In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
import pandas as pd
from scipy.optimize import minimize
from capstone_library import *

# Hints
## Sometimes Lazy is Best
You are now optimising six hyper-parameters of a machine learning model. Note that it is a popular and frequently used model, so maybe you could search to see if anyone else has optisized it before?

# Let's go!

Let's load the data.

In [2]:
X = np.load('initial_data/function_7/initial_inputs.npy')
y = np.load('initial_data/function_7/initial_outputs.npy')

In [3]:
# loading new data
new_queries = get_function_data_from_file('new_data/queries.txt', 7)
new_observ = get_function_data_from_file('new_data/observations.txt', 7)

In [4]:
# adding new_queries to X
new_queries = np.array(new_queries).reshape(-1, 6)
X = np.concatenate((X, new_queries), axis=0)

# adding new_observ to Y
new_observ = np.array(new_observ).reshape(-1)
y = np.concatenate((y, new_observ), axis=0)

## Visualizing the data and thinking of the problem

In [5]:
# visualising the data as a table
df = pd.DataFrame(np.hstack((X, y.reshape(-1, 1))), columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'y'])
df.head(100)

,x1,x2,x3,x4,x5,x6,y
0,0.272624,0.324495,0.897109,0.832951,0.154063,0.795864,0.604433
1,0.543003,0.924694,0.341567,0.646486,0.718440,0.343133,0.562753
2,0.090832,0.661529,0.065931,0.258577,0.963453,0.640265,0.007503
3,0.118867,0.615055,0.905816,0.855300,0.413631,0.585236,0.061424
4,0.630218,0.838097,0.680013,0.731895,0.526737,0.348429,0.273047
5,0.764919,0.255883,0.609084,0.218079,0.322943,0.095794,0.083747
6,0.057896,0.491672,0.247422,0.218118,0.420428,0.730970,1.364968
7,0.195252,0.079227,0.554580,0.170567,0.014944,0.107032,0.092645
8,0.642303,0.836875,0.021793,0.101488,0.683071,0.692416,0.017870
9,0.789943,0.195545,0.575623,0.073659,0.259049,0.051100,0.033565


In [6]:

# sort the data by the output, with the best value at the top
df = df.sort_values(by=['y'], ascending=False)
df.head(100)

,x1,x2,x3,x4,x5,x6,y
6,0.057896,0.491672,0.247422,0.218118,0.420428,0.730970,1.364968
41,0.000000,0.257682,0.297080,0.089549,0.438481,0.569585,1.315185
40,0.000000,0.257682,0.297080,0.089549,0.438481,0.569585,1.315185
42,0.000000,0.511413,0.153744,0.070517,0.389810,0.755719,0.787362
24,0.881647,0.204450,0.414474,0.420385,0.264915,0.730660,0.675142
14,0.148647,0.033943,0.728806,0.316066,0.021769,0.516918,0.611526
0,0.272624,0.324495,0.897109,0.832951,0.154063,0.795864,0.604433
33,0.000000,0.628959,0.757851,0.000000,0.357638,0.999999,0.582430
34,0.000000,0.628959,0.757851,0.000000,0.357638,0.999999,0.582430
36,0.000000,0.628959,0.757851,0.000000,0.357638,0.999999,0.582430


In [7]:
# Define the acquisition function to be optimized (negative UCB in this case)
def negative_acquisition(X_new, gpr, kappa):
    X_new = X_new.reshape(-1, len(X[0]))
    mean, std = gpr.predict(X_new, return_std=True)
    ucb = mean + kappa * std
    return -ucb  # we want to maximize UCB, so minimize negative UCB

def get_next_query(kappa, X, y):
    # Initialize and fit the gpr
    gpr = GaussianProcessRegressor()
    gpr.fit(X, y)

    # Define the bounds of the optimization problem, and a random initial point
    bounds = [(0, 0.999999), (0, 0.999999), (0, 0.999999), (0, 0.999999), (0, 0.999999), (0, 0.999999)]
    x0 = np.random.uniform(0, 1, size=6)  # random initialization

    # Perform the optimization using L-BFGS
    result = minimize(negative_acquisition, x0=x0, args=(gpr, kappa), bounds=bounds, method='L-BFGS-B')

    # The next query point is the one that maximizes the acquisition function
    next_query = result.x
    return next_query

In [8]:
next_query = get_next_query(1, X, y)
print(format_query(next_query))

0.000000-0.183533-0.580189-0.414292-0.428096-0.999999
